In [1]:
import numpy as np
from tqdm import tqdm
#set random seed
np.random.seed(42)

UNSHUFFLED_DECK = np.array([1,1,1,1,2,2,2,2,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], dtype=int)
PARALLEL = False #use parallel computation

def play_a_cavacamisa_game(deck, verbose=False, max_num_steps=1e5):
    """
    Play a game of cavacamisa with a given deck of cards.
    The deck is a vector of integers, with 4 possible values: 0, 1, 2, 3
    0 meaning any card which is not a 1, 2 or 3, 1 meaning a 1, 2 meaning a 2, and 3 meaning a 3.
    The deck is splitted in two and then a game is played.
    """
    #check if deck is valid
    assert np.all(np.isin(deck, [0, 1, 2, 3])), "Deck must be a vector of integers with values 0, 1, 2 or 3"
    assert len(deck) == 40, "Deck must have 40 cards"
    assert np.sum(deck == 1) == 4, "Deck must have 4 1s"
    assert np.sum(deck == 2) == 4, "Deck must have 4 2s"
    assert np.sum(deck == 3) == 4, "Deck must have 4 3s"

    #split deck in two
    d1 = list(deck[:20])
    d2 = list(deck[20:])

    # print(f'P1: {d1}')
    # print(f'P2: {d2}')

    #initialize game
    game_finished = False
    step = 0 #number of cards played
    turn = 1 #player turn, 1 or 2
    d0 = [] #cards on the table
    forced_cards = 0 # number of cards players are forced to play at a given turn (after opponent plays a 1,2,3)
    winner = None

    #play a game
    while not game_finished:
        step += 1
        if step > max_num_steps:
            game_finished = True
            print(f'Game reached maximum number of steps ({max_num_steps})')
            print(f'deck: {deck}')
        
        if turn == 1:
            #check number of cards in the player's hand
            if len(d1) == 0:
                # print(f'Player 1 has no more cards')
                game_finished = True
                winner = 2
            else:
                c = d1.pop() #play a card
                d0.append(c) #add card to the table
                
                if forced_cards == 0: #player is not forced to play cards
                    next_turn = 2 #p2 turn
                    forced_cards = c #player is forced to play c cards at the next turn

                elif forced_cards == 1:
                    if c == 0: # cant continue the chain, p2 get all cards on the table
                        d0.reverse()
                        d2 = d0 + d2 # table cards go under p2's deck
                        d0 = [] #empty table
                    next_turn = 2 #p2 turn
                    forced_cards = c

                elif forced_cards > 1:
                    if c == 0: #no forcing card, retry next turn
                        next_turn = 1
                        forced_cards -= 1
                    else: #continue forcing
                        next_turn = 2 #p2 turn
                        forced_cards = c #player is forced to play c cards at the next turn

        elif turn == 2:
            #check number of cards in the player's hand
            if len(d2) == 0:
                # print(f'Player 2 has no more cards')
                game_finished = True
                winner = 1
            else:
                c = d2.pop() #play a card
                d0.append(c) #add card to the table
                
                if forced_cards == 0: #player is not forced to play cards
                    next_turn = 1 #p1 turn
                    forced_cards = c #player is forced to play c cards at the next turn

                elif forced_cards == 1:
                    if c == 0: # cant continue the chain, p1 get all cards on the table
                        d0.reverse()
                        d1 = d0 + d1 # table cards go under p1's deck
                        d0 = [] #empty table
                    next_turn = 1 #p1 turn
                    forced_cards = c #player is not forced to play cards
                    
                elif forced_cards > 1:
                    if c == 0: #no forcing card, retry next turn
                        next_turn = 2
                        forced_cards -= 1
                    else: #continue forcing
                        next_turn = 1 #p1 turn
                        forced_cards = c #player is forced to play c cards at the next turn
        else:
            raise ValueError(f'Invalid turn: {turn}')

        if verbose:
            to_print = f' P1: {c}' + ' '*10 if turn == 1 else ' '*10 + f' P2: {c}'
            to_print = to_print + f'  {d1} {d2}  {d0}'
            # to_print = to_print + f'      {forced_cards}'
            print(to_print)
            print('-'*len(to_print)) if len(d0) == 0 else None

        turn = next_turn

    global PARALLEL
    if PARALLEL:
        return step
    else:
        return step, winner, deck
    


def play_random_cavacamisa_game(idx=0):
    if idx>0 and idx%int(1e5) == 0:
        print(f'Progress: {100*(idx/TOT_GAMES):.2f} %', flush=True)

    #shuffle deck
    deck = np.random.permutation(UNSHUFFLED_DECK)
    return play_a_cavacamisa_game(deck)


In [2]:
steps, winner, deck = play_random_cavacamisa_game()
print(f'Number of steps: {steps}')

Number of steps: 190


In [3]:
# infinite cavacamisa game
# INFINITE_DECK = np.array([0,0,3,0,2,0,2,0,3,0,3,1,0,0,0,0,0,3,1,0, 0,0,1,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,1,0])
D1 = [0,0,3,0,2,0,2,0,3,0,3,1,0,0,0,0,0,3,1,0]
D2 = [0,0,1,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,1,0]
D1.reverse()
D2.reverse()
INFINITE_DECK = np.array(D1 + D2)
print(INFINITE_DECK)
steps, winner, deck = play_a_cavacamisa_game(INFINITE_DECK, verbose=True, max_num_steps=1e3)
print(f'Infinite Game Number of steps: {steps}') 

[0 1 3 0 0 0 0 0 1 3 0 3 0 2 0 2 0 3 0 0 0 1 0 0 0 0 0 0 0 2 0 0 0 0 2 0 0
 1 0 0]
 P1: 0            [0, 1, 3, 0, 0, 0, 0, 0, 1, 3, 0, 3, 0, 2, 0, 2, 0, 3, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0]  [0]
           P2: 0  [0, 1, 3, 0, 0, 0, 0, 0, 1, 3, 0, 3, 0, 2, 0, 2, 0, 3, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0]  [0, 0]
 P1: 0            [0, 1, 3, 0, 0, 0, 0, 0, 1, 3, 0, 3, 0, 2, 0, 2, 0, 3] [0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0]  [0, 0, 0]
           P2: 0  [0, 1, 3, 0, 0, 0, 0, 0, 1, 3, 0, 3, 0, 2, 0, 2, 0, 3] [0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1]  [0, 0, 0, 0]
 P1: 3            [0, 1, 3, 0, 0, 0, 0, 0, 1, 3, 0, 3, 0, 2, 0, 2, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1]  [0, 0, 0, 0, 3]
           P2: 1  [0, 1, 3, 0, 0, 0, 0, 0, 1, 3, 0, 3, 0, 2, 0, 2, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0]  [0, 0, 0, 0, 3, 1]
 P1: 0            [0, 1, 3, 0, 0, 0, 0, 0, 1, 3, 0, 3, 0, 2, 0, 2

In [4]:
TOT_GAMES = 1e5

all_steps = []
all_winners = []
decks = []
for i in tqdm(range(int(TOT_GAMES))):
    steps, winner, deck = play_random_cavacamisa_game()
    all_steps.append(steps)
    all_winners.append(winner)
    decks.append(deck)

print(f'Average number of steps: {np.mean(all_steps)}')
print(f'Max number of steps: {np.max(all_steps)}, with deck:\n{decks[np.argmax(all_steps)]}')
print(f'Min number of steps: {np.min(all_steps)}, with deck:\n{decks[np.argmin(all_steps)]}')

print(f'Player 1 win rate: {np.sum(np.array(all_winners) == 1)/TOT_GAMES}')
print(f'Player 2 win rate: {np.sum(np.array(all_winners) == 2)/TOT_GAMES}')

100%|██████████| 100000/100000 [00:11<00:00, 9038.16it/s]


Average number of steps: 192.89238
Max number of steps: 1898, with deck:
[1 0 0 0 0 0 0 0 3 0 0 0 1 2 0 2 0 0 2 0 1 0 1 0 0 0 3 0 0 0 0 3 0 3 0 0 0
 0 0 2]
Min number of steps: 31, with deck:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1 0 0 1 0 2 0 1 0 0 0 3 3 3 2
 1 2 3]
Player 1 win rate: 0.49602
Player 2 win rate: 0.50398


In [5]:
import multiprocessing as mp
import matplotlib.pyplot as plt

TOT_GAMES = 1e9
PARALLEL = True


#initialize pool
pool = mp.Pool(10)

#play games
all_steps = pool.map(play_random_cavacamisa_game, range(int(TOT_GAMES)))
pool.close()
pool.join()

# # all_steps = list_of_rets[:,0]
# # all_winners = list_of_rets[:,1]
# # all_decks = list_of_rets[:,2]

# all_steps, all_winners, all_decks = zip(*list_of_rets)

#clear output of the cell
from IPython.display import clear_output
clear_output()

print(f'Average number of steps: {np.mean(all_steps)}')
print(f'Max number of steps: {np.max(all_steps)}')
print(f'Min number of steps: {np.min(all_steps)}')

if TOT_GAMES < 2e7:
    #plot histogram
    plt.hist(all_steps, bins=100)
    plt.show()

del all_steps

Average number of steps: 192.02061602
Max number of steps: 2769
Min number of steps: 30
